In [1]:
import numpy as np
import tensorflow as tf
import os

from input_pipeline import train_data, eval_data
from model_fns import resnet50, inceptionv4, vgg16, classification
from loss_fns  import triplet_loss_train, triplet_loss_eval, classification_loss
from loss_fns  import triplet_summary, classification_summary

!echo "GPU Device in use: '$CUDA_VISIBLE_DEVICES'"

# Feature choices
num_samples = 100000
num_eval    =   2200
batch_size  =    200

# Model for loading
model_vgg16 = \
{
    'name_out'         : 'vgg16',
    'model_fn'         : vgg16,
    'model_name'       : 'triplet_vgg16_classificaton_validation_lr0.0001_bs30_es64_normalized_exp',
    'feed_mode'        : 'triplets',
    'scope_to_restore' : 'triplet_model',
}

model_inception = \
{
    'name_out'         : 'inception',
    'model_fn'         : inceptionv4,
    'model_name'       : 'triplet_inceptionv4_classificaton_validation',
    'feed_mode'        : 'triplets',
    'scope_to_restore' : 'triplets',
}

model_resnet = \
{
    'name_out'         : 'resnet',
    'model_fn'         : resnet50,
    'model_name'       : 'triplet_resnet_classificaton_validation2layers_lr0.0001_bs30_es64_normalized_exp',
    'feed_mode'        : 'triplets',
    'scope_to_restore' : 'triplet_model',
}

model_threesplit_resnet = \
{
    'name_out'         : 'threesplit_resnet',
    'model_fn'         : resnet50,
    'model_name'       : 'triplet_threesplit_resnet_classificaton_validation2layers_lr0.0001_bs30_es64_normalized_exp',
    'feed_mode'        : 'threesplit',
    'scope_to_restore' : 'threesplit',
}

model_direct_resnet = \
{
    'name_out'         : 'direct_resnet',
    'model_fn'         : resnet50,
    'model_name'       : 'direct_resnet',
    'feed_mode'        : None,
    'scope_to_restore' : 'classifier/CNN/',
}

model_step5k_resnet = \
{
    'name_out'         : 'step5k_resnet',
    'model_fn'         : resnet50,
    'model_name'       : 'triplet_resnet_classificaton_validation2layers_lr0.0001_bs30_es64_normalized_exp',
    'feed_mode'        : 'triplets',
    'scope_to_restore' : 'triplet_model',
    'checkpoint'       : 5000
}

model_step5k_resnet = \
{
    'name_out'         : 'step5k_resnet',
    'model_fn'         : resnet50,
    'model_name'       : 'triplet_resnet_classificaton_validation2layers_lr0.0001_bs30_es64_normalized_exp',
    'feed_mode'        : 'triplets',
    'scope_to_restore' : 'triplet_model',
    'checkpoint'       : 5000
}

model_hingeloss_resnet = \
{
    'name_out'         : 'threesplit_resnet_hingeloss',
    'model_fn'         : resnet50,
    'model_name'       : 'threesplit_resnet_hingeloss_alpha1',
    'feed_mode'        : 'threesplit',
    'scope_to_restore' : 'threesplit'
}

model_step20k_hingeloss_resnet = \
{
    'name_out'         : 'step20k_threesplit_resnet_hingeloss',
    'model_fn'         : resnet50,
    'model_name'       : 'threesplit_resnet_hingeloss_alpha1',
    'feed_mode'        : 'threesplit',
    'scope_to_restore' : 'threesplit',
    'checkpoint'       : 20000
}

models = [model_step20k_hingeloss_resnet]
# models = [model_vgg16, model_inception, 
#             model_threesplit_resnet, model_resnet, 
#             model_direct_resnet, model_step5k_resnet
#             model_hingeloss_resnet, model_step20k_hingeloss_resnet]

# Folders for storage/retrival
main_directory  = '../'
checkpoints_directory = main_directory + 'checkpts/'
features_directory    = main_directory + 'features/'

GPU Device in use: '5'


In [2]:
#########
# Input #
#########

_, _, images, labels           = train_data(batch_size=batch_size, epochs=1, mode=None, small=True)
_, _, eval_images, eval_labels = eval_data (batch_size=batch_size, epochs=1, mode=None, small=True)

In [3]:
nplabels = np.load(features_directory + 'labels.npy')
npimages = np.load(features_directory + 'images.npy')

npvlabels = np.load(features_directory + 'eval_labels.npy')
npvimages = np.load(features_directory + 'eval_images.npy')

In [4]:
#################################
# Write images & labels to file #
#################################

# if not os.path.exists(features_directory):
#     os.makedirs(features_directory)

# nplabels = np.empty((0),         dtype=bytes)
# npimages = np.zeros((0,224*224), dtype=np.uint8)

# npvlabels = np.empty((0),         dtype=bytes)
# npvimages = np.zeros((0,224*224), dtype=np.uint8)

# with tf.Session() as sess: 
#     init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
#     sess.run(init_op)
      
#     num_runs = (num_samples // batch_size) + 1
#     for i in range(num_runs):
#         if (i+1) % 5 == 0: print("Step {} from {}".format(i+1, num_runs))
#         num_to_take = batch_size
#         if i+1 == num_runs: num_to_take = num_samples - i*batch_size
#         if num_to_take == 0: break

            
#         labels_out, images_out = sess.run([labels,images])
#         nplabels = np.append(nplabels, labels_out[:num_to_take], axis=0)
#         npimages = np.append(npimages, images_out[:num_to_take], axis=0)
        
#     np.save(features_directory + 'labels', nplabels)
#     np.save(features_directory + 'images', npimages)
    
#     num_runs_eval = (num_eval // batch_size) + 1
#     for i in range(num_runs_eval):
#         if (i+1) % 5 == 0: print("Step {} from {}".format(i+1, num_runs_eval))
#         num_to_take = batch_size
#         if i+1 == num_runs_eval: num_to_take = num_eval - i*batch_size
#         if num_to_take == 0: break
            
#         vlabels_out, vimages_out = sess.run([eval_labels,eval_images])
#         npvlabels = np.append(npvlabels, vlabels_out[:num_to_take], axis=0)
#         npvimages = np.append(npvimages, vimages_out[:num_to_take], axis=0)
        
#     np.save(features_directory + 'eval_labels', npvlabels)
#     np.save(features_directory + 'eval_images', npvimages)

In [5]:
image_placeholder = tf.placeholder(dtype=tf.uint8, shape=[None, 224*224])
for model in models:

    model_fn = model['model_fn']
    logits   = model_fn(image_placeholder, mode=None, name=model['scope_to_restore'])
    var_list = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=model['scope_to_restore'])
    
    assert len(var_list) > 0
    saver = tf.train.Saver(var_list=var_list)
    
    # Check validity of restoring
    with tf.Session() as sess: 
        init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
        sess.run(init_op)

        checkpoint_dir = checkpoints_directory + model['model_name'] + '/'
        # Recover previous work
        if 'checkpoint' in model:
            ckpt_path = os.path.dirname(checkpoint_dir + 'checkpoint') + '/checkpoint-' + str(model['checkpoint'])
        else: 
            ckpt = tf.train.get_checkpoint_state(os.path.dirname(checkpoint_dir + 'checkpoint'))
            if not ckpt: raise Exception("No values were restored from model {}".format(checkpoint_dir))
            ckpt_path = ckpt.model_checkpoint_path
            
        if ckpt_path:
            saver.restore(sess, ckpt_path)
            print("Values were restored")
        else: 
            raise Exception("No values were restored from model {}".format(checkpoint_dir))
            
            
        ## Save training logits  
        assert len(npimages) == num_samples
        npfeats  = np.zeros((0,64),      dtype=np.float32)
        num_runs = (num_samples // batch_size) + 1
        for i in range(num_runs):
            if (i+1) % 5 == 0: print("Step {} from {}".format(i+1, num_runs))
            num_to_take = batch_size
            if i+1 == num_runs: num_to_take = num_samples - i*batch_size
            
            start_index = i*batch_size
            end_index   = start_index + num_to_take
            
            logits_out = sess.run(logits, feed_dict={image_placeholder: npimages[start_index:end_index]})
            if len(logits_out): npfeats = np.append(npfeats, logits_out, axis=0)

        np.save(features_directory + 'features_' + model['name_out'], npfeats)

        ## Save validation logits
        assert len(npvimages) == num_eval
        npfeats_eval  = np.zeros((0,64),      dtype=np.float32)
        num_runs_eval = (num_eval // batch_size) + 1
        for i in range(num_runs_eval):
            print("Validation. Step {} from {}".format(i+1, num_runs_eval))
            num_to_take = batch_size
            if i+1 == num_runs_eval: num_to_take = num_eval - i*batch_size

            start_index = i*batch_size
            end_index   = start_index + num_to_take
            
            logits_out = sess.run(logits, feed_dict={image_placeholder: npvimages[start_index:end_index]})
            if len(logits_out): npfeats_eval = np.append(npfeats_eval, logits_out, axis=0)

        np.save(features_directory + 'eval_features_' + model['name_out'], npfeats_eval)

Instructions for updating:
keep_dims is deprecated, use keepdims instead
INFO:tensorflow:Restoring parameters from ../checkpts/threesplit_resnet_hingeloss_alpha1/checkpoint-20000
Values were restored
Step 5 from 501
Step 10 from 501
Step 15 from 501
Step 20 from 501
Step 25 from 501
Step 30 from 501
Step 35 from 501
Step 40 from 501
Step 45 from 501
Step 50 from 501
Step 55 from 501
Step 60 from 501
Step 65 from 501
Step 70 from 501
Step 75 from 501
Step 80 from 501
Step 85 from 501
Step 90 from 501
Step 95 from 501
Step 100 from 501
Step 105 from 501
Step 110 from 501
Step 115 from 501
Step 120 from 501
Step 125 from 501
Step 130 from 501
Step 135 from 501
Step 140 from 501
Step 145 from 501
Step 150 from 501
Step 155 from 501
Step 160 from 501
Step 165 from 501
Step 170 from 501
Step 175 from 501
Step 180 from 501
Step 185 from 501
Step 190 from 501
Step 195 from 501
Step 200 from 501
Step 205 from 501
Step 210 from 501
Step 215 from 501
Step 220 from 501
Step 225 from 501
Step 230 f